# 1. Introdução
O notebook abaixo apresenta e implementa o spot-checking de modelos preditivos supervisionados, desenvolvido para primeiro trabalho da disciplina Aprendizado de Máquina da Universidade Federal do Rio Grande do Sul (2024/2).

Neste trabalho, buscamos analisar a relação de diversos fatores, como gênero e notas do primeiro semestre, com a taxa de desistência de alunos. No modelo abaixo usamos o dataset carregado nesse notebook, analisamos quais os fatores que de fato influenciam na desistência dos alunos e possibilitamos que inputs personalizados sejam adicionados ao modelo para que seja calculado a probabilidade de um aluno desistir do curso.


## Setup

In [248]:
!pip install pandas plotly matplotlib seaborn scikit-learn xgboost optuna hyperopt setuptools

In [249]:
# Módulo para leitura e manipulação dos dados
import pandas as pd

# Módulo para manipulação de arrays e matrizes
import numpy as np

# Módulos para visualização de dados e plotagem de gráficos
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# Módulos específicos da sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Biblioteca com algoritmos específicos de machine learning
from xgboost import XGBClassifier

# Módulo para balanceamento de classes
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

import optuna
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# 1. Carregamento dos dados

Dataset pré-processado no notebook [t1-spot-checking.ipynb](./t1-spot-checking.ipynb)

---
Dataset obtido em https://www.kaggle.com/datasets/thedevastator/higher-education-predictors-of-student-retention/data

Original: https://zenodo.org/records/5777340#.Y7FJotJBwUE

In [250]:
data = pd.read_csv("../data/clean-dataset.csv")

In [251]:
# Separa atributos preditivos e atributo alvo
X = data.drop('Target', axis=1)
y = data['Target']

In [252]:
# Algoritmos selecionados para treinamento
dtree = DecisionTreeClassifier(random_state=0)
dtree2 = DecisionTreeClassifier(random_state=0, max_depth=10)
rfc_gini = RandomForestClassifier(random_state=2)
rfc_entropy = RandomForestClassifier(random_state=2, criterion='entropy')
lr = LogisticRegression(random_state=42)
knn_3 = KNeighborsClassifier(n_neighbors=3)
knn_5 = KNeighborsClassifier(n_neighbors=5)
abc = AdaBoostClassifier(n_estimators=50,learning_rate=1, random_state=0, algorithm='SAMME')
svmachine = svm.SVC(kernel='linear',probability=True)

algo_dict = {'Decision Tree': dtree, 'Decision Tree Max depth 5': dtree2, 'Random Forest gini': rfc_gini, 'Random Forest entropy': rfc_entropy, 'Logistic Regression': lr, '3-Nearest Neighbors': knn_3, '5-Nearest Neighbors': knn_5, 'AdaBoost': abc, 'SVM': svmachine}

In [253]:
# Referências
# https://machinelearningmastery.com/spot-check-machine-learning-algorithms-in-python/
# https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
def make_pipeline(model):
    steps = list()

    steps.append(('Normalização', StandardScaler()))
    steps.append(('Balanceamento da classe minoritária', SMOTE(sampling_strategy='minority')))
    steps.append(('Modelo', model))

    # Cria a pipeline
    pipe = Pipeline(steps=steps)

    return pipe

In [254]:
# Define hyperparameter grids for each model
def define_grid_search_params(model_name):
    if model_name == 'Random Forest gini' or model_name == 'Random Forest entropy':
        return {
            'Modelo__n_estimators': [50, 100, 150],
            'Modelo__max_depth': [10, 20, 30]
        }
    elif model_name == 'Decision Tree' or model_name == 'Decision Tree Max depth 5':
        return {
            'Modelo__max_depth': [5, 10, 20, 30]
        }
    elif model_name == 'Logistic Regression':
        return {
            'Modelo__C': [0.01, 0.1, 1, 10, 100]
        }
    elif model_name == '3-Nearest Neighbors' or model_name == '5-Nearest Neighbors':
        return {
            'Modelo__n_neighbors': [3, 5, 7, 10]
        }
    elif model_name == 'AdaBoost':
        return {
            'Modelo__n_estimators': [50, 100, 150],
            'Modelo__learning_rate': [0.01, 0.1, 1]
        }
    elif model_name == 'SVM':
        return {
            'Modelo__C': [0.01, 0.1, 1, 10],
            'Modelo__kernel': ['linear', 'rbf']
        }
    else:
        raise ValueError(f"Não foi definido parâmetro para o modelo: {model_name}")

In [255]:
def nested_cv_with_gridsearch(X, y, models, outer_folds=5, inner_folds=3, metric='f1'):
    outer_cv = StratifiedKFold(n_splits=outer_folds, shuffle=True, random_state=42)
    results = []

    for model_name, model in models.items():
        param_grid = define_grid_search_params(model_name)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            pipeline = make_pipeline(model)

            grid_search = GridSearchCV(
                estimator=pipeline,
                param_grid=param_grid,
                cv=StratifiedKFold(n_splits=inner_folds, shuffle=True, random_state=42),
                scoring=metric,
                n_jobs=-1
            )

            grid_search.fit(X_train, y_train)

            best_pipeline = grid_search.best_estimator_
            y_pred = best_pipeline.predict(X_test)

            metrics = {
                'Model': model_name,
                'F1 Score': f1_score(y_test, y_pred),
                'Precision': precision_score(y_test, y_pred),
                'Recall': recall_score(y_test, y_pred),
                'ROC AUC': roc_auc_score(y_test, best_pipeline.predict_proba(X_test)[:, 1])
            }
            results.append(metrics)

    return pd.DataFrame(results)

In [256]:
results_df = nested_cv_with_gridsearch(X, y, algo_dict, outer_folds=5, inner_folds=3, metric='f1')
display(results_df)

,Model,F1 Score,Precision,Recall,ROC AUC
0,Decision Tree,0.756014,0.740741,0.771930,0.869082
1,Decision Tree,0.763458,0.711246,0.823944,0.890403
2,Decision Tree,0.769231,0.747508,0.792254,0.892761
3,Decision Tree,0.752166,0.740614,0.764085,0.881102
4,Decision Tree,0.776173,0.796296,0.757042,0.894621
5,Decision Tree Max depth 5,0.735751,0.724490,0.747368,0.877702
6,Decision Tree Max depth 5,0.761905,0.693642,0.845070,0.892468
7,Decision Tree Max depth 5,0.767918,0.745033,0.792254,0.891618
8,Decision Tree Max depth 5,0.756014,0.738255,0.774648,0.878351
9,Decision Tree Max depth 5,0.757856,0.797665,0.721831,0.883847


In [257]:
def define_hyperopt_space(model_name):
    if model_name == 'Random Forest gini' or model_name == 'Random Forest entropy':
        return {
            'n_estimators': hp.quniform('n_estimators', 50, 300, 10),
            'max_depth': hp.quniform('max_depth', 5, 50, 1),
        }
    elif model_name == 'Decision Tree' or model_name == 'Decision Tree Max depth 5':
        return {
            'max_depth': hp.quniform('max_depth', 5, 50, 1),
        }
    elif model_name == 'Logistic Regression':
        return {
            'C': hp.loguniform('C', np.log(1e-4), np.log(1e2)),
        }
    elif model_name == '3-Nearest Neighbors' or model_name == '5-Nearest Neighbors':
        return {
            'n_neighbors': hp.quniform('n_neighbors', 3, 20, 1),
        }
    elif model_name == 'AdaBoost':
        return {
            'n_estimators': hp.quniform('n_estimators', 50, 300, 10),
            'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(2)),
        }
    elif model_name == 'SVM':
        return {
            'C': hp.loguniform('C', np.log(1e-4), np.log(1e2)),
            'kernel': hp.choice('kernel', ['linear', 'rbf']),
        }

In [258]:
def nested_cv_with_hyperopt(X, y, models, outer_folds=5, inner_folds=3, metric='f1'):
    outer_cv = StratifiedKFold(n_splits=outer_folds, shuffle=True, random_state=42)
    results = []

    for model_name, model in models.items():

        # Define objective function for Hyperopt
        def objective(params):
            # If SVM, map the kernel index to string for correct cross-validation
            if 'kernel' in params and model_name == 'SVM':
                kernel_mapping = ['linear', 'rbf']
                if isinstance(params['kernel'], int):
                    params['kernel'] = kernel_mapping[params['kernel']]

            pipeline = make_pipeline(model)

            # Convert integer-like floats to int
            final_params = {}
            for k, v in params.items():
                if isinstance(v, float) and v.is_integer():
                    v = int(v)  # Convert to int if it is a whole number float
                final_params[f'Modelo__{k}'] = v

            pipeline.set_params(**final_params)

            inner_cv = StratifiedKFold(n_splits=inner_folds, shuffle=True, random_state=42)
            scores = cross_val_score(pipeline, X_train, y_train, cv=inner_cv, scoring=metric, n_jobs=-1)
            return {'loss': -np.mean(scores), 'status': STATUS_OK}

        # Outer loop
        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            search_space = define_hyperopt_space(model_name)
            trials = Trials()

            best_params = fmin(
                fn=objective,
                space=search_space,
                algo=tpe.suggest,
                max_evals=20,
                trials=trials,
            )

            # Map SVM kernel back if needed
            if model_name == 'SVM' and 'kernel' in best_params:
                kernel_mapping = ['linear', 'rbf']
                best_params['kernel'] = kernel_mapping[best_params['kernel']]

            # Convert parameters and retrain pipeline
            final_params = {}
            for k, v in best_params.items():
                if isinstance(v, float) and v.is_integer():
                    v = int(v)
                final_params[f'Modelo__{k}'] = v

            # Train the pipeline with the best parameters
            pipeline = make_pipeline(model)
            pipeline.set_params(**final_params)
            pipeline.fit(X_train, y_train)

            # Evaluate on the test set
            y_pred = pipeline.predict(X_test)
            metrics = {
                'Model': model_name,
                'F1 Score': f1_score(y_test, y_pred),
                'Precision': precision_score(y_test, y_pred),
                'Recall': recall_score(y_test, y_pred),
                'ROC AUC': roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1])
            }
            results.append(metrics)

    return pd.DataFrame(results)

In [259]:
results_df = nested_cv_with_hyperopt(X, y, algo_dict, outer_folds=5, inner_folds=3, metric='f1')
display(results_df)

100%|██████████| 20/20 [00:17<00:00,  1.16trial/s, best loss: -0.7682731766661242]


,Model,F1 Score,Precision,Recall,ROC AUC
0,Decision Tree,0.711744,0.722022,0.701754,0.779348
1,Decision Tree,0.758065,0.699405,0.827465,0.882262
2,Decision Tree,0.772174,0.762887,0.781690,0.891935
3,Decision Tree,0.727273,0.709030,0.746479,0.837841
4,Decision Tree,0.746269,0.705329,0.792254,0.834319
5,Decision Tree Max depth 5,0.692308,0.689895,0.694737,0.763719
6,Decision Tree Max depth 5,0.774411,0.741935,0.809859,0.887992
7,Decision Tree Max depth 5,0.765432,0.766784,0.764085,0.881483
8,Decision Tree Max depth 5,0.746060,0.742160,0.750000,0.858733
9,Decision Tree Max depth 5,0.745299,0.724252,0.767606,0.847227


In [260]:
def define_optuna_space(model_name, trial):
    if model_name == 'Random Forest gini' or model_name == 'Random Forest entropy':
        return {
            'Modelo__n_estimators': trial.suggest_int('n_estimators', 50, 300),
            'Modelo__max_depth': trial.suggest_int('max_depth', 5, 50)
        }
    elif model_name == 'Decision Tree' or model_name == 'Decision Tree Max depth 5':
        return {
            'Modelo__max_depth': trial.suggest_int('max_depth', 5, 50)
        }
    elif model_name == 'Logistic Regression':
        return {
            'Modelo__C': trial.suggest_loguniform('C', 1e-4, 1e2)
        }
    elif model_name == '3-Nearest Neighbors' or model_name == '5-Nearest Neighbors':
        return {
            'Modelo__n_neighbors': trial.suggest_int('n_neighbors', 3, 20)
        }
    elif model_name == 'AdaBoost':
        return {
            'Modelo__n_estimators': trial.suggest_int('n_estimators', 50, 300),
            'Modelo__learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 2)
        }
    elif model_name == 'SVM':
        return {
            'Modelo__C': trial.suggest_loguniform('C', 1e-4, 1e2),
            'Modelo__kernel': trial.suggest_categorical('kernel', ['linear', 'rbf'])
        }

In [261]:
def nested_cv_with_optuna(X, y, models, outer_folds=5, inner_folds=3, metric='f1'):
    outer_cv = StratifiedKFold(n_splits=outer_folds, shuffle=True, random_state=42)
    results = []

    for model_name, model in models.items():
        # Define objective for Optuna
        def objective(trial):
            params = define_optuna_space(model_name, trial)

            pipeline = make_pipeline(model)
            pipeline.set_params(**params)
            inner_cv = StratifiedKFold(n_splits=inner_folds, shuffle=True, random_state=42)

            return cross_val_score(pipeline, X_train, y_train, cv=inner_cv, scoring=metric, n_jobs=-1).mean()

        # Outer loop
        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            study = optuna.create_study(direction="maximize")
            study.optimize(objective, n_trials=20)

            # Add the 'Modelo__' prefix to all best_params keys
            best_params = {f'Modelo__{key}': value for key, value in study.best_params.items()}

            pipeline = make_pipeline(model)
            pipeline.set_params(**best_params)
            pipeline.fit(X_train, y_train)

            # Evaluate on test set
            y_pred = pipeline.predict(X_test)
            metrics = {
                'Model': model_name,
                'F1 Score': f1_score(y_test, y_pred),
                'Precision': precision_score(y_test, y_pred),
                'Recall': recall_score(y_test, y_pred),
                'ROC AUC': roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1])
            }
            results.append(metrics)

    return pd.DataFrame(results)

In [262]:
results_df = nested_cv_with_optuna(X, y, algo_dict, outer_folds=5, inner_folds=3, metric='f1')
display(results_df)

[I 2024-12-07 14:09:08,081] A new study created in memory with name: no-name-2f41bba9-2ec7-4b39-9378-1e9b77af09ce
[I 2024-12-07 14:09:08,104] Trial 0 finished with value: 0.7112769948214345 and parameters: {'max_depth': 37}. Best is trial 0 with value: 0.7112769948214345.
[I 2024-12-07 14:09:08,117] Trial 1 finished with value: 0.7463663084747671 and parameters: {'max_depth': 5}. Best is trial 1 with value: 0.7463663084747671.
[I 2024-12-07 14:09:08,140] Trial 2 finished with value: 0.6960110223813238 and parameters: {'max_depth': 46}. Best is trial 1 with value: 0.7463663084747671.
[I 2024-12-07 14:09:08,163] Trial 3 finished with value: 0.6969549307480342 and parameters: {'max_depth': 36}. Best is trial 1 with value: 0.7463663084747671.
[I 2024-12-07 14:09:08,186] Trial 4 finished with value: 0.7030104284796194 and parameters: {'max_depth': 31}. Best is trial 1 with value: 0.7463663084747671.
[I 2024-12-07 14:09:08,210] Trial 5 finished with value: 0.7104404174199296 and parameters: 

,Model,F1 Score,Precision,Recall,ROC AUC
0,Decision Tree,0.747826,0.741379,0.754386,0.870558
1,Decision Tree,0.776271,0.748366,0.806338,0.891756
2,Decision Tree,0.773050,0.778571,0.767606,0.888434
3,Decision Tree,0.741652,0.740351,0.742958,0.844693
4,Decision Tree,0.765568,0.797710,0.735915,0.893119
5,Decision Tree Max depth 5,0.741818,0.769811,0.715789,0.862342
6,Decision Tree Max depth 5,0.767123,0.746667,0.788732,0.886003
7,Decision Tree Max depth 5,0.763889,0.753425,0.774648,0.888411
8,Decision Tree Max depth 5,0.755633,0.744027,0.767606,0.885853
9,Decision Tree Max depth 5,0.759777,0.806324,0.718310,0.883345
